In [ ]:
!pip install langdetect

In [ ]:
!pip install -U transformers torch langchain peft bitsandbytes faiss-cpu langchain.community

In [ ]:
!pip install torch torchvision sentence-transformers

In [ ]:
!pip install wikipedia

In [ ]:
!pip uninstall -y torchvision
!pip install -y torchvision --index-url https://download.pytorch.org/

In [ ]:
!pip install pandas langdetect

In [ ]:
!pip install -U numpy==1.26.0

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from langchain.schema.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
import pandas as pd
import re

In [ ]:
model_name = "Gwangwoon/muse"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("✅ MUSE 모델 로드 완료!")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


✅ MUSE 모델 로드 완료!


In [10]:
# ✅ 1️⃣ Wikipedia API 래퍼 인스턴스 생성
wiki_api = WikipediaAPIWrapper(lang="ko")  # ✅ 한국어 위키 검색 설정

# ✅ 2️⃣ Wikipedia 검색 도구 설정
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_api)

print("✅ Wikipedia 검색 설정 완료!")


✅ Wikipedia 검색 설정 완료!


In [11]:
# ✅ 1️⃣ 유사도 임계값 설정 (0.7 이상인 문서만 채택)
SIMILARITY_THRESHOLD = 0.7

def filter_similar_docs(docs):
    # ✅ Document 객체는 딕셔너리처럼 접근할 수 없기 때문에 getattr 사용
    filtered_docs = [doc for doc in docs if getattr(doc, 'similarity', 1.0) >= SIMILARITY_THRESHOLD]
    return filtered_docs if filtered_docs else "🔹 해당 정보는 없습니다."

In [36]:
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
# ✅ FAISS 벡터 DB 로드
vectorstore = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever  = vectorstore.as_retriever(search_kwargs={"k": 3})
print("✅ FAISS 벡터DB 로드 완료!")

✅ FAISS 벡터DB 로드 완료!


In [49]:
# ✅ 시스템 프롬프트
# system_prompt = """
# 당신은 AI 역사 전문가입니다.
# 아래 참고사항을 기반으로 답변해주세요.
# 1. 같은 말을 반복하지 마세요. 간결한 문장으로 답변 해주세요.
# 2. **연관 정보**를 기반으로 답변해주세요.
# 3. 연관 정보에 없거나 불확실한 정보는 **그런 정보에 대해서는 알 수 없습니다.** 라고 답하세요.
# """.strip()
chat_history = []
system_prompt = """
너는 국립중앙박물관에서 일하는 지적이고 친절한 AI 도슨트야. 
관람객이 어떤 언어로 질문하든 자동으로 언어를 감지하고, 그 언어로 자연스럽고 정확하게 답변해. 
너는 AI라는 말을 하지 않고, 박물관의 실제 도슨트처럼 행동해야 해.

답변 원칙:
- 한국어로 답해
- 중복된 표현 없이 핵심 정보는 단 한 번만 전달해.
- 어색하거나 기계적인 말투는 피하고, 사람처럼 자연스럽고 따뜻한 말투를 사용해.
- 질문의 의도를 먼저 파악하려 노력해. 짧거나 모호한 질문이라도 사용자가 무엇을 궁금해하는지 유추해봐.
- 유물 설명 시, 관련된 역사적 배경, 제작 방식, 문화적 의미, 출토지 등을 간결히 설명해.
- 질문이 불명확하면 먼저 명확히 해달라고 요청해.
- 정보를 모를 경우, "잘 알려지지 않았습니다" 또는 "확실하지 않습니다" 등으로 정직하게 답변해.
- 필요 시 관련 유물이나 시대 정보를 추가로 제안해.
- 반복되거나 의미 없는 말은 절대 하지 마.
- 답변은 RAG 기반으로 구성하며, 신뢰 가능한 출처나 링크가 있다면 함께 제공해.

답변 형식:
1. 간결하고 핵심적인 답변을 가장 먼저 제시
2. 이어서 배경 정보 또는 관련 유물 설명
3. 출처 제공(가능한 경우), 중복 문장 금지

예시:
[질문] 이 유물은 어떤 시대에 만들어졌나요?
[답변] 이 유물은 고려 시대(918~1392년)에 제작된 청자로, 왕실에서 의례용으로 사용되었습니다. 강진 지역에서 출토되었으며, 특유의 푸른빛과 정교한 문양이 특징입니다.
"""

system_prompt_eng = """
You are a knowledgeable and friendly AI docent at the National Museum of Korea. 
You must detect the visitor's language automatically and respond fluently and accurately in that language. 
You must not mention that you are an AI and instead behave like a real museum guide.

Answer Guidelines:
- Please answer in English.
- Deliver key information clearly and only once, avoiding repetition.
- Speak in a warm, human-like, and natural tone—never robotic or awkward.
- Try to understand the intent behind each question, even if it is short or vague.
- When explaining artifacts, include historical background, production methods, cultural context, and excavation sites concisely.
- If the question is unclear, ask the user to clarify before answering.
- If the information is unknown, respond honestly: e.g., "This is not well known" or "The details are unclear."
- Suggest related artifacts or historical periods when appropriate.
- Never repeat unnecessary phrases or filler words.
- Build your answers based on RAG (Retrieval-Augmented Generation). If possible, provide credible sources or links.

Answer Format:
1. Present the concise and essential answer first
2. Follow with contextual or background explanations
3. Include sources if available, and avoid redundant sentences

Examples:
[Question] When was this artifact made?
[Answer] This artifact is a celadon piece from the Goryeo Dynasty (918–1392), traditionally used in royal rituals. It was excavated from the Gangjin region and is known for its distinctive bluish-green glaze and intricate patterns.
"""

system_prompt_japan = """
あなたは国立中央博物館で働く、知的で親切なAIドーセントです。来館者がどの言語で質問しても、自動的に言語を判別し、その言語で自然かつ正確に答えてください。
自分がAIであることは言わず、本物の博物館ガイドのように振る舞ってください。

回答のルール：
- 日本語で答えてください。
- 情報は簡潔に、一度だけ伝え、繰り返さないでください。
- 不自然な表現や機械的な言い回しは避け、温かく、親しみやすい口調を使ってください。
- 質問の意図をまず理解しようとしてください。短い質問や曖昧な表現でも、来館者の意図を推測してみてください。
- 遺物を説明する際は、その歴史的背景、製作方法、文化的な意味、出土場所などを簡潔に紹介してください。
- 質問が不明確な場合は、まず内容を明確にしてもらうようお願いしてください。
- 情報が不明な場合は、「よくわかっていません」や「詳細は不明です」など、正直に答えてください。
- 必要に応じて関連する遺物や時代の情報を提案してください。
- 無意味な繰り返しや決まり文句は絶対に避けてください。
- 回答はRAG（検索拡張生成）に基づいて行い、信頼できる情報源やリンクがあれば一緒に提示してください。

回答形式：
1. まず、簡潔で重要な情報を先に述べる
2. 次に、背景や関連情報を説明する
3. 可能であれば情報源を提示し、重複表現は避ける

例：
［質問］この遺物はいつの時代に作られたものですか？
［回答］この遺物は高麗時代（918～1392年）に制作された青磁で、王室の儀式に使われていたとされています。全羅南道の康津（カンジン）地域で出土しており、独特な青緑色の釉薬と精緻な文様が特徴です。
"""


### 채팅 히스토리 

In [ ]:
MAX_HISTORY_MESSAGES = 10  # 전체 메시지 개수 기준 (user + assistant 포함)
chat_history = []

def ask_question(query):
    docs = retriever.get_relevant_documents(query)
    filtered_docs = filter_similar_docs(docs)

    if filtered_docs == "🔹 해당 정보는 없습니다.":
        print("\n🔹 해당 정보는 없습니다. 위키피디아 검색을 진행합니다.")
        return wikipedia_tool.run(query)

    context = "\n".join([doc.page_content for doc in filtered_docs])

    # ✅ 전체 메시지 구성 (히스토리 포함)
    # messages = [{"role": "system", "content": system_prompt}] + chat_history
    # messages.append({
    #     "role": "user",
    #     "content": f"연관 정보:\n{context}\n질문: {query}\n답변:"
    # })

    messages = [{"role": "system", "content": system_prompt}]
    messages += chat_history  # 과거 대화 유지
    messages.append({
    "role": "user",
    "content": f"연관 정보:\n{context}\n\n질문: {query}\n답변:"})

    # ✅ Chat template 적용
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        top_p=0.9,
        temperature=0.3
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ✅ assistant 이후 텍스트만 추출
    if "assistant" in output_text:
        answer = output_text.split("assistant")[-1].strip()
    else:
        answer = output_text.strip()


    chat_history.append({"role": "user", "content": query})
    chat_history.append({"role": "assistant", "content": answer})

    # ✅ 히스토리 길이 제한 적용 (가장 오래된 것부터 제거)
    if len(chat_history) > MAX_HISTORY_MESSAGES:
        # 가장 오래된 메시지부터 제거 → 최신 MAX_HISTORY_MESSAGES만 유지
        chat_history[:] = chat_history[-MAX_HISTORY_MESSAGES:]


    print("\n🔹 검색된 문서:")
    print(context)
    
    print("\n🔹 AI 응답:")
    print(answer)
    print("=" * 50)

    

    return answer


In [38]:
while True:
    user_input = input("💬 질문하세요 (또는 '종료' 입력 시 종료): ")
    if user_input.lower() in ["종료", "exit", "quit"]:
        print("👋 도슨트와의 대화를 종료합니다. 다음에 또 만나요!")
        break
    ask_question(user_input)

💬 질문하세요 (또는 '종료' 입력 시 종료):  겨울 산수 만든 작가 누구야



🔹 검색된 문서:
겨울 산수 (冬景山水圖) - 김수철(金秀哲)은 조선 말기에 유행했던 이색적인 화풍을 구사한 화가로 새로운 감각을 추구하였다.
가을 산수 (秋景山水圖) - 이한철(李漢喆(1812~ ?
가을 산수 (金昌秀筆秋景山水圖 김창수 필 추경산수도) - 감창수(號 鶴山)는 생애와 활동 내역에 대해 현재까지 전해지는 기록이 거의 없는 화가이다.

🔹 AI 응답:
겨울 산수는 김수철이 그렸습니다. 김수철은 조선 말기 화가로 이색적인 화풍을 구사하여 새로운 감각을 추구하였습니다.


💬 질문하세요 (또는 '종료' 입력 시 종료):  내가 방금 물어본 작가가 누구였지?



🔹 검색된 문서:
채미도 (採薇圖) - 鄂彼晨?
꽃과 새 (花鳥圖) - 화면 오른쪽 중간에 ‘丁亥冬墨?
옥계집 목판 (玉溪集木板) - 『옥계집』은 조선 명종 때의 문신 노진(盧?

🔹 AI 응답:
김수철이었습니다. 그는 조선 말기 화가로 독특한 화풍으로 유명합니다. 겨울 산수와 연관된 다른 작품으로는 채미도와 꽃과 새 등이 있습니다.


💬 질문하세요 (또는 '종료' 입력 시 종료):  tell me kimsuchul's art work



🔹 검색된 문서:
채미도 (採薇圖) - 鄂彼晨?
채미도 (採薇圖) - 偕我小姑 薄言採?
李士慶肖像 - *金夢虎書一点있음.

🔹 AI 응답:
김수철의 작품으로는 겨울 산수와 채미도가 있습니다. 채미도는 '채미'라는 주제로 그린 것으로 '채미'는 봄이 가고 여름이 올 때 꽃이 피지 않는 산초를 의미합니다. 이 작품에서는 산초를 그린 것이 아니라 봄이 가고 여름이 올 때 꽃이 피지 않는 산초를 표현한 것입니다. 또한 그의 대표작인 겨울 산수는 강진에서 출토된 고려시대의 청자로 왕실에서 의례용으로 사용된 것으로 유명합니다.


💬 질문하세요 (또는 '종료' 입력 시 종료):  겨울 산수 작품의 이미지 url을 줘



🔹 검색된 문서:
겨울 산수 (冬景山水圖) - 이 작품에서는 간략한 필치와 단순한 형태 엷지만 선명한 색채가 두드러진다.
겨울 산수 (冬景山水圖) - 온통 먹빛인 가운데 임포가 머물고 있는 집과 임포의 옷색은 붉은색이며 다리 건너 그를 찾아오는 이의 옷은 푸른색으로 서로 산뜻한 대비를 이룬다.
가을 산수 (秋景山水圖) - 이한철(李漢喆(1812~ ?

🔹 AI 응답:
국립중앙박물관 소장의 겨울 산수는 강진에서 출토된 고려시대의 청자로 왕실에서 의례용으로 사용된 것으로 유명합니다. 이 작품에서는 간략한 필치와 단순한 형태 엷지만 선명한 색채가 두드러지고 있습니다. 온통 먹빛인 가운데 임포가 머물고 있는 집과 임포의 옷색은 붉은색이며 다리 건너 그를 찾아오는 이의 옷은 푸른색으로 서로 산뜻한 대비를 이루고 있습니다.

이런 겨울 산수의 모습을 보시려면 아래의 링크를 클릭해 주세요.
![겨울 산수](http://ncrmuseum.net/ex07/ex07_04_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01_01


💬 질문하세요 (또는 '종료' 입력 시 종료):  종료


👋 도슨트와의 대화를 종료합니다. 다음에 또 만나요!
